# Automated ML

Import all the dependencies that will be needed for the project.

In [42]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset


print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


# Workspace

In [43]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-134454
aml-quickstarts-134454
southcentralus
610d6e37-4747-4a20-80eb-3aad70a55f43


## Dataset

### Overview
We will be using a dataset about hear disease from [kaggle](https://www.kaggle.com/ronitf/heart-disease-uci)

* age: age in years 
* sex: (1 = male; 0 = female) 
* cp: chest pain type
* trestbps: resting blood pressure (in mm Hg on admission to the hospital)
* chol: serum cholestoral in mg/dl 
* fbs: (fasting blood sugar &gt; 120 mg/dl) (1 = true; 0 = false) 
* restecg: resting electrocardiographic results 
* thalach: maximum heart rate achieved 
* exang: exercise induced angina (1 = yes; 0 = no) 
* oldpeak: ST depression induced by exercise relative to rest 
* slope: the slope of the peak exercise ST segment
* ca: number of major vessels (0-3) colored by flourosopy 
* thal: 3 = normal; 6 = fixed defect; 7 = reversable defect 
* target: 1 or 0 


Accessing the dataset:

In [44]:
found = False
key = "heart-disease-dataset"
description_text = "Heart disease dataset"

if key in ws.datasets.keys():
        print("Dataset {} already exists..!".format(key))
        found = True
        dataset = Dataset.get_by_name(ws, name=key)

if not found:
        # Create AML Dataset and register it into Workspace
        print("Creating {} dataset..".format(key))
        data = 'https://raw.githubusercontent.com/SadmiB/Heart-Disease-Prediction/master/heart.csv'
        dataset = Dataset.Tabular.from_delimited_files(data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

Dataset heart-disease-dataset already exists..!


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00,303.00
mean,54.37,0.68,0.97,131.62,246.26,0.15,0.53,149.65,0.33,1.04,1.40,0.73,2.31,0.54
std,9.08,0.47,1.03,17.54,51.83,0.36,0.53,22.91,0.47,1.16,0.62,1.02,0.61,0.50
min,29.00,0.00,0.00,94.00,126.00,0.00,0.00,71.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,47.50,0.00,0.00,120.00,211.00,0.00,0.00,133.50,0.00,0.00,1.00,0.00,2.00,0.00
50%,55.00,1.00,1.00,130.00,240.00,0.00,1.00,153.00,0.00,0.80,1.00,0.00,2.00,1.00
75%,61.00,1.00,2.00,140.00,274.50,0.00,1.00,166.00,1.00,1.60,2.00,1.00,3.00,1.00
max,77.00,1.00,3.00,200.00,564.00,1.00,2.00,202.00,1.00,6.20,2.00,4.00,3.00,1.00


In [66]:
dataset.take(-5).to_pandas_dataframe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.30,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.50,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.40,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.80,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.60,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.20,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.20,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.40,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.20,1,1,3,0


# Experiment

In [46]:
# choose a name for experiment
experiment_name = 'heart-disease-experiment-ml1'
project_folder = 'heart-disease-project'
experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
heart-disease-experiment-ml1,quick-starts-ws-134454,Link to Azure Machine Learning studio,Link to Documentation


# Create or Attach an AmlCompute cluster

In [47]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.


## AutoML Configuration

Below the Automl settings and cofiguration we using AUC_weighted as a primary metric and the task to solve is classification. The target column is "target" which represents whether a patient has a heart disease or not.

In [48]:
# Automl settings
automl_settings = { 
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# automl config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="target",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [49]:
# Submit the ml experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [50]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [51]:
remote_run.wait_for_completion()

{'runId': 'AutoML_a44c0534-d182-4428-b34f-1a1e4ddb03c6',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-12T15:29:28.629048Z',
 'endTimeUtc': '2021-01-12T15:56:34.629892Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-disease-experiment-ml1","subscription_id":"610d6e37-4747-4a20-80eb-3aad70a55f43","resource_group":"aml-quickstarts-134454","workspace_name":"quick-starts-ws-134454","region":"southcentralus","compute_target":"auto-ml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":

## Best Model

Getting the best model from the automl experiments and display all the properties of the model.



In [52]:
import joblib
# Retrieve and save your best automl model.

best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('- Accuracy:', best_run_metrics['accuracy'])
print('- AUC_weighted:', best_run_metrics['AUC_weighted'])
joblib.dump(fitted_model,'azureml_fitted_model.sav')

Best Run Id:  AutoML_a44c0534-d182-4428-b34f-1a1e4ddb03c6_46
- Accuracy: 0.8611827956989249
- AUC_weighted: 0.9190213390124381


['azureml_fitted_model.sav']

### Register best model

In [53]:
model_name = best_run.properties["model_name"]
description = "AutoML model for predetcting heart disease."
tags = None
model = remote_run.register_model(model_name=model_name, description=description, tags=tags)

print(remote_run.model_id)

AutoMLa44c0534d46


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [54]:
script_file_name = 'inference/score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

### Deploy the model as a webservice using ACI

In [69]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "bmData", 'type': "automl_classification"}, 
                                               description = 'Heart Disease prediction using Automl Classification')

aci_service_name = 'automl-heart-disease-prediction'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl-heart-disease-prediction
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


### Test the web service

In [73]:
#Test 1
import requests
import json

data = '''{ 
  "data": [  
    { 
      "age": 63,
      "sex": 1,
      "cp": 3,
      "trestbps": 145,
      "chol": 233,
      "fbs": 1,
      "restecg": 0,
      "thalach": 150,
      "exang": 0,
      "oldpeak": 2.3,
      "slope": 0,
      "ca": 0,
      "thal": 1
    }
  ]
}'''

headers = {'Content-Type': 'application/json'}

resp = requests.post(aci_service.scoring_uri, data, headers=headers)

pred = json.loads(json.loads(resp.text))['result']
print(pred)

[1]


In [74]:
# Test 2
import requests
import json

data = '''{ 
  "data": [  
    { 
      "age": 57,
      "sex": 0,
      "cp": 0,
      "trestbps": 140,
      "chol": 241,
      "fbs": 0,
      "restecg": 1,
      "thalach": 123,
      "exang": 1,
      "oldpeak": 0.2,
      "slope": 1,
      "ca": 0,
      "thal": 3
    }
  ]
}'''

headers = {'Content-Type': 'application/json'}

resp = requests.post(aci_service.scoring_uri, data, headers=headers)

pred = json.loads(json.loads(resp.text))['result']
print(pred)

[0]


### Get service logs

In [57]:
aci_service.get_logs()

'2021-01-12T15:59:34,981429300+00:00 - gunicorn/run \n2021-01-12T15:59:34,988160700+00:00 - iot-server/run \n2021-01-12T15:59:34,987325400+00:00 - rsyslog/run \n2021-01-12T15:59:35,019080300+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

### Delete the web service

In [65]:
aci_service.delete()

End!